In [ ]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext  
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils   

sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
# ssc = StreamingContext(sc, 5)
ssc = StreamingContext(sc, 5)


In [ ]:
directKafkaStream = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'localhost:9092'})

In [ ]:
lines = directKafkaStream.map(lambda x: x[1])

In [ ]:
import re

In [ ]:
# Lazily instantiated global instance of SparkSession
def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

# DataFrame operations inside your streaming program

words = lines # DStream of strings

def process(time, rdd):
    print("========= %s =========" % str(time))
    try:
        print("Get the singleton instance of SparkSession")
        spark = getSparkSessionInstance(rdd.context.getConf())

        print("Convert RDD[String] to RDD[Row] to DataFrame")
        rowRdd = rdd.map(lambda w: Row(word=w))
        wordsDataFrame = spark.createDataFrame(rowRdd)

        print("Creates a temporary view using the DataFrame")
        wordsDataFrame.createOrReplaceTempView("words")

        print("Do word count on table using SQL and print it")
        wordCountsDataFrame = spark.sql("select word, count(*) as total from words group by word")
        wordCountsDataFrame.show()
    except Exception, e:
        print(str(e))
        pass

words.foreachRDD(process)

In [ ]:
ssc.start()  
ssc.awaitTermination()




In [ ]:
ssc.stop()
sc.stop()